# Adding particles using NASA JPL Horizons system

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Adding-particles" data-toc-modified-id="Adding-particles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adding particles</a></span></li><li><span><a href="#Time" data-toc-modified-id="Time-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Time</a></span></li><li><span><a href="#Reference-Frame" data-toc-modified-id="Reference-Frame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference Frame</a></span></li></ul></div>

REBOUND can add particles to simulations by obtaining ephemerides from NASA's powerful HORIZONS database.  HORIZONS supports many different options, and we will certainly not try to cover everything here.  This is meant to serve as an introduction to the basics, beyond what's in [Churyumov-Gerasimenko.ipynb](Churyumov-Gerasimenko.ipynb).  If you catch any errors, or would either like to expand on this documentation or improve REBOUND's HORIZONS interface (`rebound/horizons.py`), please do fork the repository and send us a pull request.

## Adding particles

When we add particles by passing a string, REBOUND queries the HORIZONS database and takes the first dataset HORIZONS offers.  For the Sun, moons, and small bodies, this will typically return the body itself.  For planets, it will return the barycenter of the system (for moonless planets like Venus it will say barycenter but there is no distinction).  If you want the planet specifically, you have to use, e.g., "NAME=Pluto" rather than "Pluto".  In all cases, REBOUND will print out the name of the HORIZONS entry it's using.

You can also add bodies using their integer NAIF IDs:  [NAIF IDs](http://naif.jpl.nasa.gov/pub/naif/toolkit_docs/MATLAB/req/naif_ids.html).  Note that because of the number of small bodies (asteroids etc.) we have discovered, this convention only works for large objetcts.  For small bodies, instead use "NAME=name" (see the SMALL BODIES section in the [HORIZONS Documentation](http://ssd.jpl.nasa.gov/?horizons_doc)).

In [1]:
import rebound
sim = rebound.Simulation()
sim.add("Sun")
## Other examples:
# sim.add("Venus")
# sim.add("399")
# sim.add("Europa")
# sim.add("NAME=Ida")
# sim.add("Pluto")
# sim.add("NAME=Pluto")
sim.status()

Searching NASA Horizons for 'Sun'... Found: Sun (10).
---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	May 18 2019 03:15:47
Number of particles: 	1
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0022425022101477004 y=0.007617108185657758 z=-1.889041100733308e-05 vx=-0.0004889123496614232 vy=2.485328822895337e-06 vz=1.2722738672369346e-05>
---------------------------------


Currently, HORIZONS does not have any mass information for solar system bodies.  `rebound/horizons.py` has a hard-coded list provided by Jon Giorgini (10 May 2015) that includes the planets, their barycenters (total mass of planet plus moons), and the largest moons.  If REBOUND doesn't find the corresponding mass for an object from this list (like for the asteroid Ida above), it will print a warning message.  If you need the body's mass for your simulation, you can set it manually, e.g. (see [Units.ipynb](Units.ipynb) for an overview of using different units):

In [2]:
sim.add("NAME=Ida")
print(sim.particles[-1])    # Ida before setting the mass
sim.particles[-1].m = 2.1e-14 # Setting mass of Ida in Solar masses
print(sim.particles[-1])    # Ida after setting the mass

Searching NASA Horizons for 'NAME=Ida'... Found: 243 Ida.
<rebound.Particle object, m=0.0 x=2.393511477901243 y=-1.630786508025444 z=0.001978052927709383 vx=0.30860319350571613 vy=0.4937893483930798 vz=0.011473449719916774>
<rebound.Particle object, m=2.1e-14 x=2.393511477901243 y=-1.630786508025444 z=0.001978052927709383 vx=0.30860319350571613 vy=0.4937893483930798 vz=0.011473449719916774>


/home/colin/anaconda3/envs/ml/lib/python3.6/site-packages/rebound/horizons.py:140: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


## Time

By default, REBOUND queries HORIZONS for objects' current positions.  Specifically, it caches the current time the first time you call `rebound.add`, and gets the corresponding ephemeris.  All subsequent calls to `rebound.add` will then use that initial cached time to make sure you get a synchronized set of ephemerides.

You can also explicitly pass REBOUND the time at which you would like the particles ephemerides:  

In [3]:
sim = rebound.Simulation()
date = "2005-06-30 15:24" # You can also use Julian Days. For example:   date = "JD2458327.500000"
sim.add("Venus")
sim.add("Venus", date=date)
sim.status()

Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	May 18 2019 03:15:47
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=2.4478382877847715e-06 x=0.2571965263283425 y=0.6803627724733973 z=-0.005759568913553802 vx=-1.1014001664384536 vy=0.41761518415744453 vz=0.06927368656604146>
<rebound.Particle object, m=2.4478382877847715e-06 x=-0.6612393630774994 y=0.26970405120283636 z=0.04197291272901321 vx=-0.4456694249946371 vy=-1.0953912972697604 vz=0.010725840920973641>
---------------------------------


We see that the two Venus positions are different.  The first call cached the current time, but since the second call specified a date, it overrode the default.  Any time you pass a date, it will overwrite the default cached time, so: 

In [4]:
sim = rebound.Simulation()
date = "2005-06-30 15:24"   # You can also use Julian Days. For example:   date = "JD2458327.500000"
sim.add("Venus", date=date)
sim.add("Earth")
sim.status()

Searching NASA Horizons for 'Venus'... Found: Venus Barycenter (299).
Searching NASA Horizons for 'Earth'... Found: Earth-Moon Barycenter (3).
---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	May 18 2019 03:15:47
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=2.4478382877847715e-06 x=-0.6612393630774994 y=0.26970405120283636 z=0.04197291272901321 vx=-0.4456694249946371 vy=-1.0953912972697604 vz=0.010725840920973641>
<rebound.Particle object, m=3.040432648022642e-06 x=0.14670980646665963 y=-0.9980850744551467 z=2.6690120061396023e-05 vx=0.9725640752593823 vy=0.14274934345662696 vz=3.869931888850666e-06>
---------------------------------


would set up a simulation with Venus and Earth, all synchronized to 2005-06-30 15:24.  All dates should either be passed in the format Year-Month-Day Hour:Minute or in JDxxxxxxx.xxxx for a date in Julian Days.

REBOUND takes these absolute times to the nearest minute, since at the level of seconds you have to worry about exactly what time system you're using, and small additional perturbations probably start to matter.  For reference HORIZONS interprets all times for ephemerides as [Coordinate (or Barycentric Dynamical) Time](https://en.wikipedia.org/wiki/Barycentric_Dynamical_Time).

## Reference Frame

REBOUND queries for particles' positions and velocities relative to the Sun:

In [5]:
sim = rebound.Simulation()
sim.add("Sun")
sim.status()

Searching NASA Horizons for 'Sun'... Found: Sun (10).
---------------------------------
REBOUND version:     	3.8.2
REBOUND built on:    	May 18 2019 03:15:47
Number of particles: 	1
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0022425022101477004 y=0.007617108185657758 z=-1.889041100733308e-05 vx=-0.0004889123496614232 vy=2.485328822895337e-06 vz=1.2722738672369346e-05>
---------------------------------


The reference plane is the ecliptic (Earth's orbital plane) of J2000 (Jan. 1st 2000 12:00 GMT), with the x axis along the ascending node of the ecliptic and the Earth's mean equator (also at J2000).  